<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/EM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [2]:
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  print(i)
  train = train.append(i)



      Unnamed: 0                                           Sentence    Target
0            659  Appellant had stated to the officers that she ...   Invalid
1           3456  We shall discuss the facts more fully in conne...    Others
2           2043  â€œPerjury is a false statement, either writte...   Invalid
3           3344  The offense is felony theft by false pretext; ...     Issue
4           3231  Numerous contentions urging the commission of ...     Issue
...          ...                                                ...       ...
3471        1561  Therefore, the deletion of reference to the Cl...  Analysis
3472         327  A paraffin test, which was made on the appella...  Analysis
3473        1462  The sole question presented is the sufficiency...     Facts
3474        1169  The answer was in response to the question ask...     Facts
3475        1470  The Stateâ€™s evidence shows that in the sprin...   Invalid

[3476 rows x 3 columns]
      Unnamed: 0                       

In [3]:
train.sort_values("Sentence", inplace = True)
print(len(train))


37711


In [4]:
 new_train = train.drop_duplicates(subset ="Sentence")


In [5]:
train = new_train

In [6]:
train['Target'].unique()

array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Others',
       'Conclusion', 'Issue'], dtype=object)

In [7]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()


array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Conclusion',
       'Issue'], dtype=object)

In [8]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(4416, 7374)


In [10]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [ ]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

unlabel_1 = unlabel.loc[:100]


def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  #print(unlabel_2.head())
  return unlabel_2

unlabel_1 = index_reset(unlabel_1)
unlabel_1_copy = unlabel_1



x_un1 = tfidf_vect.transform(unlabel_1['text'])






In [ ]:
unlabel_1['Target']=-1

In [ ]:
import numpy as np
np.unique(unlabel_1['Target'])

In [ ]:
train = train.rename(columns={'Sentence':'text'})

In [ ]:
 from sklearn.preprocessing import LabelEncoder
 train['Target']= LabelEncoder().fit_transform(train['Target'])

In [ ]:
train_and_unlabel =  pd.concat([train,unlabel_1])
print(len(train))
print(len(unlabel_1))
print(len(train_and_unlabel))

In [ ]:
train_and_unlabel.head()

In [ ]:

train_and_unlabel.reset_index(inplace=True)

In [ ]:
train_and_unlabel.columns

In [ ]:
del train_and_unlabel['index']
del train_and_unlabel['Unnamed: 0']

In [ ]:
train_and_unlabel.head()

In [ ]:
train_and_unlabel.tail()

In [ ]:
train_and_unlabel_vect = tfidf_vect.transform(train_and_unlabel['text'])

In [ ]:
train_and_unlabel_vect_df=pd.DataFrame(train_and_unlabel_vect.toarray())


In [ ]:
train_and_unlabel_vect_df.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_classifier = GaussianNB()

gnb_classifier.fit(train_and_unlabel_vect_df, train_and_unlabel['Target'])

In [ ]:
test_pred = gnb_classifier.predict(t_p.toarray())

In [ ]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)

In [ ]:
unlabel_to_classify = pd.DataFrame(x_un1.toarray())
unlabel_predict = gnb_classifier.predict(unlabel_to_classify)
len(unlabel_predict)

In [ ]:
np.unique(unlabel_predict)

In [ ]:
type(unlabel_predict)

In [ ]:
count = 0
total = 0 
out_pos = []
pos = []
for i in unlabel_predict:
  #print(i)
  if i == -1:
    pos.append(count)
    total = total + 1
  else:
    out_pos.append(count)

  count = count + 1
  

    



In [ ]:
print(count)
print(total)

In [ ]:
out_pos

In [ ]:
print(pos)

In [ ]:

#unlabel_1 = unlabel_1.loc[pos,:]
RN = unlabel_1.loc[pos,:]

In [ ]:
len(RN)

In [ ]:
type(RN)

In [ ]:
RN.columns

In [ ]:
Q =  unlabel_1.loc[out_pos,:]

In [ ]:
len(Q)

In [ ]:
type(Q)

In [ ]:
type(Q)

In [ ]:
#comment once increases unlabel data
Q = unlabel_1

In [ ]:
unlabel_1.head()

In [ ]:
RN['new_Target'] = unlabel_predict

In [ ]:
RN.head()

In [ ]:
del RN['new_Target']

In [ ]:
del train1
del train2
del train3
del train4
del train5
del train6
del train7
del train8
del train9
del train10
del train_highKappa

In [ ]:
#RN = unlabel_1

In [ ]:
type(unlabel_1)

In [ ]:
type(RN)

In [ ]:
RN.columns

In [ ]:
train.columns

In [ ]:
del train['Unnamed: 0']

In [ ]:
#Step 2:

In [ ]:
# p_and_RN = pd.concat([train,RN])

In [ ]:
# p_and_RN.reset_index(inplace=True)

In [ ]:
# p_and_RN.columns

In [ ]:
# p_and_RN.head()

In [ ]:
# del p_and_RN['index']

In [ ]:
# p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])

In [ ]:
# p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())


In [ ]:
# type(unlabel_1)

In [ ]:
# type(RN)

In [ ]:
/

In [ ]:
# Q_vect = tfidf_vect.transform(Q['text'])

In [ ]:
# Q_vect_df=pd.DataFrame(Q_vect.toarray())


In [ ]:
# import lightgbm as lgb
# lgb_classifier = lgb.LGBMClassifier()
# lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])

In [ ]:
# np.unique(p_and_RN['Target'])

In [ ]:
# Q_pred = lgb_classifier.predict(Q_vect_df)

In [ ]:
# np.unique(Q_pred)

In [ ]:
# count_q = 0
# total_q = 0 
# out_pos_q = []
# pos_q = []
# for i in Q_pred:
#   #print(i)
#   if i == -1:
#     pos_q.append(count_q)
#     total_q = total_q + 1
#   else:
#     out_pos_q.append(count_q)

#   count_q = count_q + 1

In [ ]:
# W = Q.loc[pos_q,:]

In [ ]:
# if W.empty :
#   pass
# else:
#   Q = Q.loc[total_q,:]
#   RN = pd.concat([RN,W])

In [ ]:
#defina Q for loop1

In [ ]:
loop_variable = 0

In [ ]:
# Q = unlabel_1

In [ ]:
#Q.columns

In [ ]:
import lightgbm as lgb
for i in range(0,3):
  print(loop_variable)
  p_and_RN = pd.concat([train,RN])
  p_and_RN.reset_index(inplace=True)
  p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
  p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

  Q_vect = tfidf_vect.transform(Q['text'])
  Q_vect_df=pd.DataFrame(Q_vect.toarray())

  lgb_classifier = lgb.LGBMClassifier()
  lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
  np.unique(p_and_RN['Target'])

  Q_pred = lgb_classifier.predict(Q_vect_df)
  np.unique(Q_pred)

  count_q = 0
  total_q = 0 
  out_pos_q = []
  pos_q = []
  for i in Q_pred:
    #print(i)
    if i == -1:
      pos_q.append(count_q)
      total_q = total_q + 1
    else:
      out_pos_q.append(count_q)

    count_q = count_q + 1

  Q.reset_index(inplace=True)
  W = Q.loc[pos_q,:]


  if W.empty :
    print("W is empty, came out of loop")
    break
  else:
    Q_new = Q.loc[out_pos_q,:]
    Q = Q_new
    RN = pd.concat([RN,W])
    RN.reset_index(inplace=True)
    loop_variable = loop_variable + 1
  print("completed iteration")

In [ ]:
type(pos_q)

In [ ]:
Q.loc[pos_q,:]

In [ ]:
Q

In [ ]:
Q.reset_index(inplace=True)

In [ ]:
Q

In [ ]:
Q[1]

In [ ]:
Q_pred

In [ ]:
len(pos_q)

In [ ]:
test_pred =  lgb_classifier.predict(t_p.toarray())

In [ ]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)